# Surgery

Purpose: Move all the weight and biases from MikeNet to TensorFlow

In [ ]:
import os
os.chdir('/home/jupyter/triangle_model/')

In [ ]:
import troubleshooting, meta, modeling, data_wrangling
import numpy as np
import tensorflow as tf
import pandas as pd
import altair as alt

from ipywidgets import interact

# Load model from disk

In [ ]:
code_name = "surgery_v10"
cfg = meta.ModelConfig.from_json(f'models/{code_name}/model_config.json')
model = modeling.MyModel(cfg)
model.build()
model.load_weights(f'{cfg.weight_folder}/ep0001')

In [ ]:
# Re-Double check the weights looks like what we put in
import matplotlib.pyplot as plt

def my_heatmap(x):
    plt.imshow(x, cmap='hot', interpolation='none')
    plt.colorbar()

[my_heatmap(w) for w in model.weights if w.name.startswith("w_hop_oh")]

# Intact triangle model input temporal dynamics
Need to fix Diagnosis class... somehow not really working... check plot input manually

In [ ]:
ts = data_wrangling.load_testset('mn_r100')

model.set_active_task('triangle')
y_pred = model([ts['ort']] * cfg.n_timesteps)


## Collect time invarying elements

In [ ]:
layer = 'sem'
target_word = 'close'
target_word_idx = ts['item'].index(target_word)
bias_name = 'bias_s'

df_dict = {}
df_dict["target_act"] = ts[layer][target_word_idx, :]
df_dict["bias"] = [w.numpy() for w in model.weights if w.name.startswith(bias_name)][0]
df_time_invar = pd.DataFrame.from_dict(df_dict)
df_time_invar["unit"] = df_time_invar.index
df_time_invar["word"] = target_word
df_time_invar

## Collect time varying elements

In [ ]:
SEM_NAME_MAP = {
    "input_hps_hs": "PS",
    "input_css_cs": "CS",
    "input_sem_ss": "SS",
    "input_hos_hs": "OS",
    "input_sem": "input",
    "sem": "act",
}
PHO_NAME_MAP = {
    "input_hsp_hp": "SP",
    "input_cpp_cp": "CP",
    "input_pho_pp": "PP",
    "input_hop_hp": "OP",
    "input_pho": "input",
    "pho": "act",
}

name_map = SEM_NAME_MAP

df_time_varying = pd.DataFrame()

for i, model_output_name in enumerate(name_map.keys()):
    this_output_df = pd.DataFrame()
    for t in range(13):
        df_dict = {}
        name = name_map[model_output_name]
        df_dict[name] = y_pred[model_output_name][t, target_word_idx, :]
        this_step_df = pd.DataFrame.from_dict(df_dict)
        this_step_df["timetick"] = t
        this_step_df["unit"] = this_step_df.index
        this_output_df = pd.concat([this_output_df, this_step_df], ignore_index=True)

    if i == 0:
        df_time_varying = this_output_df
    else:
        df_time_varying = pd.merge(
            df_time_varying, this_output_df, on=["timetick", "unit"]
        )

df_time_varying

In [ ]:
# Merge and export
df = df_time_varying.merge(df_time_invar, on="unit", how="left")
df["unit_acc"] = abs(df.target_act - df.act) < 0.5
df = df[
    ["word", "unit", "unit_acc", "timetick", "target_act", "bias"]
    + list(name_map.values())
]

# Restructure
melt_value_vars = ["bias"] + list(name_map.values())
df = df.melt(
    id_vars=["word", "unit", "timetick", "target_act", "unit_acc"],
    value_vars=melt_value_vars,
)

In [ ]:
all_on_nodes = df.loc[df.target_act == 1, "unit"].unique()
all_off_nodes = df.loc[df.target_act == 0, "unit"].unique()
print(f"On: {all_on_nodes} \nOff: {np.random.choice(all_off_nodes, 10)}")

In [ ]:
sel_node = all_on_nodes
node_df = df.loc[df.unit.isin(sel_node)]
plot_df = node_df.loc[~node_df.variable.isin(['act', 'input', 'SS', 'PP'])]

alt.Chart(plot_df).mark_line().encode(
    y='mean(value):Q', 
    x='timetick:Q', 
    color='variable:N'
    ).properties(title=f"word: {target_word} at nodes: {sel_node}")

# Accuracy at the end of training

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def min_cosine_distance_idx(all_reps, pred):
    """return the index of word that has min cosine distance"""
    all_cosine_dist = [cosine(pred, rep) for rep in all_reps]
    return np.argmin(all_cosine_dist) 

def cosine_accuracy(all_reps, pred, target):
    """Check whether the prediction is the min cosine distance word"""
    target_idx = min_cosine_distance_idx(all_reps, target)
    pred_idx = min_cosine_distance_idx(all_reps, pred)
    return target_idx == pred_idx

def all_cosine_accuracy(all_reps, preds, targets):
    return np.mean([cosine_accuracy(all_reps, pred, target) for pred, target in zip(preds, targets)])

def binary_accuracy(pred, target):
    """Calculate correct side accuracy"""
    d = abs(pred - target)
    max_d = np.max(d, axis = 1)
    # print(max_d)
    return np.mean(max_d < 0.5)

def get_all_acc(y_pred):
    """Calculate accuracy of all outputs"""
    mn_train = data_wrangling.load_testset('mn_train')
    sem_acc = all_cosine_accuracy(all_reps=mn_train['sem'], preds=y_pred['sem'][-1].numpy(), targets=ts['sem'])
    pho_acc = binary_accuracy(ts['pho'], y_pred['pho'][-1, :, :].numpy())
    return {
        "pho": pho_acc,
        "sem": sem_acc
    }

def get_task_acc(task):
    """Get task acc"""
    model.set_active_task(task)
    input_name = modeling.IN_OUT[task][0]
    y_pred = model([ts[input_name]] * cfg.n_timesteps)
    return get_all_acc(y_pred)


In [ ]:
get_task_acc('triangle')

In [ ]:
get_task_acc('ort_sem')

In [ ]:
get_task_acc('exp_ops')

In [ ]:
get_task_acc('exp_osp')

In [ ]:
get_task_acc('ort_pho')

# Lesion

In [ ]:
model.set_active_task('ort_sem')
y_pred_os = model([ts['ort']] * cfg.n_timesteps)
get_all_acc(y_pred_os)

In [ ]:
model.set_active_task('exp_ops')
y_pred_os = model([ts['ort']] * cfg.n_timesteps)
get_all_acc(y_pred_os)